In [ ]:
import langchain
from langchain import PromptTemplate, LLMChain
from langchain.llms import TextGen, OpenAI
import subprocess
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

question = "Why is the sky blue?"
num_variations = 5

In [ ]:
def OpenAi_Chain(question, num_variations):
    
    llm = OpenAI(openai_api_key = OPENAI_API_KEY,
                 temperature = 0.3,
                 model_name = "text-davinci-003")
    
    template = """
    Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instruction: Your task is to generate {num_variations} different unique versions of the given user question. 
    Generate these versions from different perspectives and in different tones.  
    Please take the given question, make these variations and then answer each question 
    variation in a unique way based off its subtle differences from the others. 
    please ONLY return the ANSWERS you generate NOT the questions, and make sure you generate ALL {num_variations} anwsers separated by new lines.

    Input:{question}

    Response:
    """

    prompt = PromptTemplate(template=template, input_variables=["question", "num_variations"])

    llm_chain = LLMChain(prompt=prompt, llm=llm)

    Model_Response = llm_chain({'question': question, 'num_variations': num_variations})
    Answer_List = Model_Response["text"]
    return Answer_List

In [ ]:
def local_llama(question, num_variations):
    cmd = ["python", "/home/ghrist/text-generation-webui/server.py", "--model", "Llama-2-7b-hf", "--lora", "test-4bit", "--api"]
    process = subprocess.Popen(cmd, cwd="/home/ghrist/text-generation-webui")

    langchain.debug = True

    model_url = "http://localhost:5000"

    template = """
    Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instruction: Your task is to generate {num_variations} different unique versions of the given user question. 
    Generate these versions from different perspectives and in different tones.  
    Please take the given question, make these variations and then answer each question 
    variation in a unique way based off its subtle differences from the others. 
    please ONLY return the ANSWERS you generate NOT the questions, and make sure you generate ALL {num_variations} anwsers separated by new lines.

    Input:{question}

    Response:
    """

    prompt = PromptTemplate(template=template, input_variables=["question", "num_variations"])
    llm = TextGen(model_url=model_url)
    llm_chain = LLMChain(prompt=prompt, llm=llm)

    Model_Response = llm_chain({'question': question, 'num_variations': num_variations})
    Answer_List = Model_Response["text"]
    process.terminate()

    return Answer_List

In [ ]:
def Answer_Consolidator_Chain(All_Answers):

    template = """  
    I will provide you with a list of answers, all addressing the same question but in slightly varying ways. 
    Your task is to analyze and consolidate these answers into one cohesive and representative response. 
    In your consolidation, prioritize the common consensus found in the majority of the answers. 
    If certain answers deviate or introduce unique points, weigh them less in favor of the prevailing sentiment or information presented by the majority. 
    Produce a single, ultimate answer that best represents the collective input, controlling for any outliers or randomness.
    The list of answers to perform this on are as follows: {All_Answers}
    """ 
    prompt_template = PromptTemplate(
        input_variables=["All_Answers"], 
        template=template
    )

    llm = OpenAI(
        openai_api_key=OPENAI_API_KEY,
        temperature=0.3,
        max_tokens=500,
        model_name="text-davinci-003"
        )

    consolidator_chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

    response = consolidator_chain({All_Answers})
    
    return response["text"]

In [ ]:
########TEST########

tv1 = local_llama(question, num_variations)

tv2 = OpenAi_Chain(question, num_variations)

both_answers = tv1 + "\n" + tv2

Answer_Consolidator_Chain(both_answers)